# SROS Example: From Random Structures to Structures with Specified Short-Range Order (Using HE-DRX as an example)

This notebook demonstrates how to use the code in this repository to complete the following steps:：

1. Generate a random disordered rocksalt (DRX) structure with a specified chemical composition；
2. Use calculation.SROS.SRO to adjust the F-Li and Li-Li short-range order (SRO) parameters；
3. Use calculation.SROS.Ewald to further reduce the Coulomb electrostatic energy；
4. Use calculation.analysis to perform basic analysis of the local environments in the structure.

> Before running this notebook, please ensure that dependencies such as pymatgen, smol, monty, and scipyare installed.



In [ ]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import sys
import numpy as np

from pymatgen.core import Structure

from calculation.generate_random import make_supercell_matrix, modify_structure_HEDRX
from calculation.SROS import SRO, Ewald
from calculation.analysis import (
    check_coordination_crystalnn,
    calculate_bond_lengths_crystalnn,
)


In [ ]:
# 1. 生成 NaCl 型超胞
# 原代码 scaling_factors=(2, 4, 5) 对应对角矩阵
scaling_matrix = [
    [2, 0, 0],
    [0, 4, 0],
    [0, 0, 5]
]

supercell = make_supercell_matrix(scaling_matrix=scaling_matrix)
print("--- Supercell Info ---")
print(supercell.formula)
print("Number of sites (Na+Cl):", supercell.num_sites)

# 2. 按 TM 方案随机替换为 Li / TM / O / F
tm_type = "TM4"  # 可选: "TM2", "TM4", "TM6"
random_seed = 123

# 调用 modify_structure_HEDRX 进行替换
drx_random = modify_structure_HEDRX(supercell, tm_type=tm_type, seed=random_seed)

print("\n--- Substituted Structure Info ---")
print("Composition:", drx_random.composition)

# 3. 保存结果
out_dir = Path("./examples")
out_dir.mkdir(exist_ok=True)
random_path = out_dir / f"random_{tm_type}.vasp"

# 导出为 POSCAR
drx_random.to(filename=str(random_path), fmt="poscar")

print(f"\nRandom DRX structure written to: {random_path}")

# 可选：快速检查元素比例是否符合预期
from collections import Counter
species = [site.species_string for site in drx_random]
print("Element counts:", Counter(species))

--- Supercell Info ---
Na40 Cl40
Number of sites (Na+Cl): 80

--- Substituted Structure Info ---
Composition: Li26 Mn8 Ti2 Nb4 O34 F6

Random DRX structure written to: examples/random_TM4.vasp
Element counts: Counter({'O': 34, 'Li': 26, 'Mn': 8, 'F': 6, 'Nb': 4, 'Ti': 2})


In [ ]:
# 使用 SRO 类调节 F–Li 与 Li–Li 短程有序参数

sro = SRO(
    structure_path=str(random_path),
    anion_a="F",
    anion_b="O",
    cation="Li",
    c_cation=26 / 40,
)

print("Initial alpha_F-Li:", sro.a)
print("Initial alpha_Li-Li:", sro.a_LiLi)

target_alpha_FLi = 0.2
target_alpha_LiLi = 0.0
tol = 0.05
max_steps = 5000
rate = 1.0

sro.run(
    max_steps=max_steps,
    target_alpha=target_alpha_FLi,
    target_alpha_LiLi=target_alpha_LiLi,
    rate=rate,
    tol=tol,
    random_seed=42,
)

print("Final alpha_F-Li:", sro.a)
print("Final alpha_Li-Li:", sro.a_LiLi)

sro_tuned_path = out_dir / f"sro_tuned_{tm_type}.vasp"
sro.to_file(str(sro_tuned_path))
print("SRO-tuned structure written to:", sro_tuned_path)


Initial alpha_F-Li: 0.10256410256410264
Initial alpha_Li-Li: 0.03353057199211051
Innitial alpha: 0.10256410256410264 Innitial alphaLiLi: 0.03353057199211051
No exchange
Steps： 0 New alpha: 0.10256410256410264
No exchange
Steps： 1 New alpha: 0.10256410256410264
No exchange
Steps： 2 New alpha: 0.10256410256410264
No exchange
Steps： 3 New alpha: 0.10256410256410264
No exchange
Steps： 4 New alpha: 0.10256410256410264
No exchange
Steps： 5 New alpha: 0.10256410256410264
No exchange
Steps： 6 New alpha: 0.10256410256410264
Less neighboring
Steps： 7 New alpha: 0.10256410256410264
No exchange
Steps： 8 New alpha: 0.10256410256410264
No exchange
Steps： 9 New alpha: 0.10256410256410264
No exchange
Steps： 10 New alpha: 0.10256410256410264
Less neighboring
Steps： 11 New alpha: 0.14529914529914537
No exchange
Steps： 12 New alpha: 0.14529914529914537
No exchange
Steps： 13 New alpha: 0.14529914529914537
More neighboring
Steps： 14 New alpha: 0.14529914529914537
Less neighboring
Steps： 15 New alpha: 0.188

In [ ]:
# 可选：基于 Ewald 电静能进行进一步优化

out_ewald_path = out_dir / f"sro_ewald_{tm_type}.vasp"

ewald = Ewald(
    structure_path=str(sro_tuned_path),
    out_path=str(out_ewald_path),
    tm_type=tm_type,
)

print("Ewald-optimized structure written to:", out_ewald_path)


The supercell size for the processor is 1 prims.
The ensemble has a total of 80 sites.
The active sublattices are:


Sampling 1 chain(s) from a cell with 80 sites: 100%|██████████| 100000/100000 [00:02<00:00, 34234.89it/s]


Ewald-optimized structure written to: examples/sro_ewald_TM4.vasp


In [ ]:
# 结构检查与简单分析

final_struct_path = out_ewald_path if out_ewald_path.exists() else sro_tuned_path
final_struct = Structure.from_file(str(final_struct_path))

print("Final structure composition:", final_struct.composition)
print("Number of sites:", final_struct.num_sites)

print("\n=== Check Li coordination (CrystalNN, expected CN=6) ===")
li_abnormal = check_coordination_crystalnn(
    structure_file=final_struct,
    element_symbol="Li",
    expected_cn=6,
)

if not li_abnormal:
    print("All Li sites have CN = 6 (by CrystalNN).")

print("\n=== Li bond length statistics (CrystalNN) ===")
_, flattened_lengths, avg_length = calculate_bond_lengths_crystalnn(
    structure_file=final_struct,
    element_symbol="Li",
)

print(f"Total Li bonds: {len(flattened_lengths)}")
print(f"Average Li-ligand bond length: {avg_length:.4f} Å")


Final structure composition: F6 Li26 Mn8 Nb4 O34 Ti2
Number of sites: 80

=== Check Li coordination (CrystalNN, expected CN=6) ===
Found 26 Li atoms
All Li sites have CN = 6 (by CrystalNN).

=== Li bond length statistics (CrystalNN) ===


/opt/anaconda3/envs/pymatge/lib/python3.12/site-packages/pymatgen/analysis/local_env.py:4021: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  r1 = _get_radius(structure[n])
/opt/anaconda3/envs/pymatge/lib/python3.12/site-packages/pymatgen/analysis/local_env.py:4023: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  r2 = _get_radius(entry["site"])
/opt/anaconda3/envs/pymatge/lib/python3.12/site-packages/pymatgen/analysis/local_env.py:3935: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  nn_data = self.get_nn_data(structure, n)


Total bonds: 156
Average bond length: 2.1000 Å
Total Li bonds: 156
Average Li-ligand bond length: 2.1000 Å


## Summary

This notebook demonstrates a typical workflow for using the SROS code：

1. Using `calculation.generate_random` with `make_supercell` and `substitute_elements`，
   to construct a random DRX structure with the target chemical composition.
2. Using `calculation.SROS.SRO` to calculate and adjust the Warren-Cowley type short-range order parameter   $\alpha_{F-Li}$ 与 $\alpha_{Li-Li}$ ；
3. Using `calculation.SROS.Ewald` to further optimize the atomic site occupancy based on Ewald electrostatic energy；
4. Using `calculation.analysis` to perform quantitative analysis of the local coordination environment and bond lengths.。

You can modify the parameters in this example based on your own system (different TM combinations, target α values, or supercell sizes) to generate a series of controlled SRO structures for subsequent DFT or other analyses.